In [ ]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc
from shutil import copy,SameFileError
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from qiskit_ibm_provider import IBMProvider

from quantum.Quantum import QuantumRegressor
from quantum.Evaluate import evaluate
from settings import ANSATZ_LIST, ENCODER_LIST
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import InconsistentVersionWarning

In [ ]:
def data_gen(datadirs):
    dfs=[]
    for i in datadirs:
        with open(i,'r') as g:
            data = json.load(g)
        df = pd.DataFrame.from_dict(data)
        name = os.path.basename(os.path.dirname(os.path.dirname(i)))
        df['pairs']=os.path.basename(os.path.dirname(i))
        if "RUD"  in name:
            df['RUD']=int(name.split('_')[0])
            df['AL']=1
            # df['RUD'] = 
        elif "AL" in name:
            df['RUD']=1
            df['AL']=int(name.split('_')[0])
        else:
            df['RUD']=1
            df['AL']=1
        dfs.append(df)
        
    stacked = pd.concat(dfs)
    return stacked

In [ ]:
rud_al=glob("*_lin16qubit/*/*results.json")

base_names = []
for i in glob('5_AL_lin16qubit/*/*results.json'):
    name = os.path.basename(os.path.dirname(i))
    base_names.append(f"../sixteenqubit_test/lin16qubit/{name}/{name}_results.json")

lindf = data_gen(base_names+rud_al)

In [ ]:
data = lindf.rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['pairs','AL','RUD'], value_vars=['Train','Test'])

# Get unique pairs
unique_pairs = data["pairs"].unique()



for i, pair in enumerate(unique_pairs):  # Limit to 3 sets of 5 subplots
    # Define subplots
    fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True)
    axes = axes.flatten()
    subset = data[data["pairs"] == pair]
    
    # Left subplot (x-axis: AL)
    sns.lineplot(data=subset, x="AL", y="value", hue="variable", ax=axes[0], marker="o")
    axes[0].set_title(f"{pair} (AL)")
    axes[0].set_xlabel("AL")
    axes[0].set_ylim(0,1)
    # Right subplot (x-axis: RUD)
    
    sns.lineplot(data=subset, x="RUD", y="value", hue="variable", ax=axes[1], marker="o")
    axes[1].set_title(f"{pair} (RUD)")
    axes[1].set_xlabel("RUD")

    plt.tight_layout()
    plt.show()

In [ ]:
unique_pairs.shape

In [ ]:
melteddf = lindf.rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['AL','RUD'], value_vars=['Train','Test'])

fig, (ax1,ax2) = plt.subplots(1,2,sharey=True,figsize=(10,5))
sns.lineplot(melteddf[melteddf['RUD']==1],x='AL',y='value',hue='variable',ax=ax1,palette=sns.color_palette('Paired',2),marker='o', markeredgecolor='black')
sns.lineplot(melteddf[melteddf['AL']==1],x='RUD',y='value',hue='variable',ax=ax2,palette=sns.color_palette('Paired',2),marker='o', markeredgecolor='black')
ax1.set_ylim(0,1)
ax1.set_ylabel("R$^{2}$")
ax1.set_xlim(0.75,5.25)
ax2.set_xlim(0.75,5.25)
ax1.set_xlabel("Ansatz Layers (AL)")
ax2.set_xlabel("Re-upload Depth (RUD)")
# Remove legends from individual plots
ax1.get_legend().remove()
ax2.get_legend().remove()

# Global legend
plt.legend(loc="upper right", bbox_to_anchor=(1.5, 1.0))
plt.tight_layout()

In [ ]:
z